In [1]:
import pandas as pd

In [2]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from feature_engine.imputation import DropMissingData

from sklearn import set_config
set_config(display="diagram")
set_config(transform_output="pandas")
import numpy as np
import pprint
import pandas as pd

In [3]:
df = pd.read_csv("../data/dataset_raw/banking.csv")

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


In [8]:
ls = []
for index, val in enumerate(df.columns.to_list()):
    ls.append(f"{index} : {val}")

In [10]:
ls

['0 : age',
 '1 : job',
 '2 : marital',
 '3 : education',
 '4 : default',
 '5 : housing',
 '6 : loan',
 '7 : contact',
 '8 : month',
 '9 : day_of_week',
 '10 : duration',
 '11 : campaign',
 '12 : pdays',
 '13 : previous',
 '14 : poutcome',
 '15 : emp_var_rate',
 '16 : cons_price_idx',
 '17 : cons_conf_idx',
 '18 : euribor3m',
 '19 : nr_employed',
 '20 : y']

In [5]:
cols = ['job',
  'marital',
  'education',
  'default',
  'housing',
  'loan',
  'contact',
  'month',
  'day_of_week',
  'poutcome']

In [6]:

feature_dict = {}
for feature in cols:
    idict = {}
    for index, item in enumerate(df[f'{feature}'].unique().tolist()):
#     print(f'{item} : {index}')
        idict[index+1] = item
        feature_dict[feature] = idict
pprint.pprint(feature_dict)

{'contact': {1: 'cellular', 2: 'telephone'},
 'day_of_week': {1: 'thu', 2: 'fri', 3: 'tue', 4: 'mon', 5: 'wed'},
 'default': {1: 'unknown', 2: 'no', 3: 'yes'},
 'education': {1: 'basic.4y',
               2: 'unknown',
               3: 'university.degree',
               4: 'high.school',
               5: 'basic.9y',
               6: 'professional.course',
               7: 'basic.6y',
               8: 'illiterate'},
 'housing': {1: 'yes', 2: 'no', 3: 'unknown'},
 'job': {1: 'blue-collar',
         2: 'technician',
         3: 'management',
         4: 'services',
         5: 'retired',
         6: 'admin.',
         7: 'housemaid',
         8: 'unemployed',
         9: 'entrepreneur',
         10: 'self-employed',
         11: 'unknown',
         12: 'student'},
 'loan': {1: 'no', 2: 'yes', 3: 'unknown'},
 'marital': {1: 'married', 2: 'single', 3: 'divorced', 4: 'unknown'},
 'month': {1: 'aug',
           2: 'nov',
           3: 'jun',
           4: 'apr',
           5: 'jul',
   

In [7]:
df.marital.unique()

array(['married', 'single', 'divorced', 'unknown'], dtype=object)

In [8]:
df.education.unique()

array(['basic.4y', 'unknown', 'university.degree', 'high.school',
       'basic.9y', 'professional.course', 'basic.6y', 'illiterate'],
      dtype=object)

In [9]:
df.default.unique() # - Ordinal

array(['unknown', 'no', 'yes'], dtype=object)

In [10]:
df.housing.unique() # - Ordinal

array(['yes', 'no', 'unknown'], dtype=object)

In [11]:
df.loan.unique()  # - Ordinal Encode

array(['no', 'yes', 'unknown'], dtype=object)

In [12]:
df.contact.unique() # - OneHot

array(['cellular', 'telephone'], dtype=object)

In [13]:
df.month.unique() # - Ordinal

array(['aug', 'nov', 'jun', 'apr', 'jul', 'may', 'oct', 'mar', 'sep',
       'dec'], dtype=object)

In [14]:
df.day_of_week.unique() # - Ordinal

array(['thu', 'fri', 'tue', 'mon', 'wed'], dtype=object)

In [15]:
df.poutcome.unique() # - Ordinal

array(['nonexistent', 'success', 'failure'], dtype=object)

In [16]:
# trans = ColumnTransformer(
#         [('ordinal Encoder - poutcome', OrdinalEncoder(
#             categories=[['nonexistent', 'success', 'failure']],
#             handle_unknown='use_encoded_value',
#             unknown_value=-1,), ['poutcome']),
         
#          ('ordinal Encoder - Day of weel', OrdinalEncoder(
#             categories=[['mon', 'tue', 'wed', 'thu', 'fri']],
#             handle_unknown='use_encoded_value',
#             unknown_value=-1,), ['day_of_week']),
         
#          ('ordinal Encoder - loan', OrdinalEncoder(
#             categories=[['no', 'yes', 'unknown']],
#             handle_unknown='use_encoded_value',
#             unknown_value=-1,), ['loan']),
         
#         ('OneHot - contact', OneHotEncoder(sparse_output=False, 
#                                            handle_unknown='ignore'), ['contact'])
        
        
#         ], remainder='drop', verbose_feature_names_out=False
# )

In [17]:
# df_transed = trans.fit_transform(df)

In [18]:
# trans.transformers_

In [19]:
# df_transed

In [20]:
#  ('ordinal Encoder - Day of weel', OrdinalEncoder(
#             categories=[['mon', 'tue', 'wed', 'thu', 'fri']],
#             handle_unknown='use_encoded_value',
#             unknown_value=-1,), ['day_of_week']),

# ('ordinal Encoder - loan', OrdinalEncoder(
#             categories=[['no', 'yes', 'unknown']],
#             handle_unknown='use_encoded_value',
#             unknown_value=-1,), ['loan']),

### Preprocessing Pipeline

In [13]:
encode = {'contact': {1: 'cellular', 2: 'telephone'},
 'day_of_week': {1: 'thu', 2: 'fri', 3: 'tue', 4: 'mon', 5: 'wed'},
 'default': {1: 'unknown', 2: 'no', 3: 'yes'},
 'education': {1: 'unknown',
               2: 'illiterate',
               3: 'basic',
               4: 'high.school',
               5: 'university.degree',           
               6: 'professional.course',
               },
 'housing': {1: 'yes', 2: 'no', 3: 'unknown'},
 'job': {1: 'blue-collar',
         2: 'technician',
         3: 'management',
         4: 'services',
         5: 'retired',
         6: 'admin.',
         7: 'housemaid',
         8: 'unemployed',
         9: 'entrepreneur',
         10: 'self-employed',
         11: 'unknown',
         12: 'student'},
 'loan': {1: 'no', 2: 'yes', 3: 'unknown'},
 'marital': {1: 'married', 2: 'single', 3: 'divorced', 4: 'unknown'},
 'month': {1: 'mar',
           2: 'apr',
           3: 'may',
           4: 'jun',
           5: 'jul',
           6: 'aug',
           7: 'sep',
           8: 'oct',
           9: 'nov',
           10: 'dec'},
 'poutcome': {1: 'nonexistent', 2: 'success', 3: 'failure'}}

In [14]:
def edu_trans(df):
    df['education'] = df['education'].replace(['basic.4y', 'basic.6y', 'basic.9y'], 'basic')
    return df 

In [15]:
ls

['0 : age',
 '1 : job',
 '2 : marital',
 '3 : education',
 '4 : default',
 '5 : housing',
 '6 : loan',
 '7 : contact',
 '8 : month',
 '9 : day_of_week',
 '10 : duration',
 '11 : campaign',
 '12 : pdays',
 '13 : previous',
 '14 : poutcome',
 '15 : emp_var_rate',
 '16 : cons_price_idx',
 '17 : cons_conf_idx',
 '18 : euribor3m',
 '19 : nr_employed',
 '20 : y']

In [16]:
cat_trans = ColumnTransformer(
        [         
          ('ordinal Encoder', OrdinalEncoder(
            categories=[list(encode['day_of_week'].values()),
                        list(encode['education'].values()),
                        list(encode['job'].values()),
                        list(encode['marital'].values()),
                        list(encode['month'].values()),
                       ],
            handle_unknown='use_encoded_value',
            unknown_value=-1,), 
          [9, 3, 1, 2, 8]), #['day_of_week', 'education', 'job', 'marital', 'month']
        
         
        ('OneHot Encoder', OneHotEncoder(sparse_output=False, 
                                           handle_unknown='ignore'), 
         [7, 4, 5, 6, 14])   # ['contact', 'default', 'housing', 'loan', 'poutcome']
        
        
        ], remainder='passthrough', verbose_feature_names_out=False
)

In [19]:
cat_trans

ColumnTransformer(remainder='passthrough',
                  transformers=[('ordinal Encoder',
                                 OrdinalEncoder(categories=[['thu', 'fri',
                                                             'tue', 'mon',
                                                             'wed'],
                                                            ['unknown',
                                                             'illiterate',
                                                             'basic',
                                                             'high.school',
                                                             'university.degree',
                                                             'professional.course'],
                                                            ['blue-collar',
                                                             'technician',
                                                             'management',
                                                             'services',
                                                             'retired',
                                                             'admin.',
                                                             'housemaid',
                                                             'unemployed',
                                                             'entrepreneur',
                                                             'self-employed',
                                                             'unknown',
                                                             'student'],
                                                            ['married',
                                                             'single',
                                                             'divorced',
                                                             'unknown'],
                                                            ['mar', 'apr',
                                                             'may', 'jun',
                                                             'jul', 'aug',
                                                             'sep', 'oct',
                                                             'nov', 'dec']],
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 [9, 3, 1, 2, 8]),
                                ('OneHot Encoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 [7, 4, 5, 6, 14])],
                  verbose_feature_names_out=False)

In [20]:
ls

['0 : age',
 '1 : job',
 '2 : marital',
 '3 : education',
 '4 : default',
 '5 : housing',
 '6 : loan',
 '7 : contact',
 '8 : month',
 '9 : day_of_week',
 '10 : duration',
 '11 : campaign',
 '12 : pdays',
 '13 : previous',
 '14 : poutcome',
 '15 : emp_var_rate',
 '16 : cons_price_idx',
 '17 : cons_conf_idx',
 '18 : euribor3m',
 '19 : nr_employed',
 '20 : y']

In [21]:
num_col = ColumnTransformer(
    
    [
        ('Scaling Numerical features', StandardScaler(), [0, 10, 11, 12, 13, 15, 16, 17, 18, 19]),
        
        
    ],  
            
            remainder='passthrough', verbose_feature_names_out=False)     # ['age','duration','campaign', 'pdays','previous','emp_var_rate','cons_price_idx','cons_conf_idx','euribor3m','nr_employed']

In [22]:
num_col

ColumnTransformer(remainder='passthrough',
                  transformers=[('Scaling Numerical features', StandardScaler(),
                                 [0, 10, 11, 12, 13, 15, 16, 17, 18, 19])],
                  verbose_feature_names_out=False)

In [23]:
# DropMissingData()

In [24]:
pipe = Pipeline(
    [   
        # ('DropMissingValues', DropMissingData()),
        # ('handling - education feature', FunctionTransformer(edu_trans)),
        ('categorical Transformer', cat_trans),
        ('Numerical Transformer', num_col)
        
    ]
)

In [25]:
from sklearn.model_selection import train_test_split 

X = df.drop('y', axis=1)
y = df['y']

In [26]:
X.shape

(41188, 20)

In [27]:
pipe = pipe.fit(X)

In [45]:
data

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,...,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0.381527,-0.186230,-0.565922,0.195414,-0.349494,0.839061,-0.227465,0.951267,0.773575,0.845170,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.245157,-0.463926,-0.565922,0.195414,-0.349494,-0.115781,-0.649003,-0.323542,0.230456,0.398115,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,-1.153816,0.311309,0.156105,-5.117342,3.691766,-1.134279,0.828107,0.151810,-1.667578,-2.428157,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,-0.098268,-0.282652,-0.204909,0.195414,-0.349494,-1.197935,-0.864955,-1.425496,-1.277824,-0.940281,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.437075,-0.467783,-0.565922,-5.133393,1.671136,-1.898153,-2.374889,1.966794,-1.586859,-1.257233,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,1.820911,-0.139947,-0.565922,0.195414,-0.349494,0.839061,1.536429,-0.280328,0.717649,0.845170,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
41184,-0.865939,-0.240227,-0.204909,0.195414,-0.349494,0.648092,0.722722,0.886447,0.714190,0.331680,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
41185,0.189609,-0.757050,0.156105,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
41186,0.765363,-0.224799,-0.204909,0.195414,-0.349494,-2.216433,-1.977538,2.939106,-1.660082,-2.069683,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [28]:
pipe

Pipeline(steps=[('categorical Transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal Encoder',
                                                  OrdinalEncoder(categories=[['thu',
                                                                              'fri',
                                                                              'tue',
                                                                              'mon',
                                                                              'wed'],
                                                                             ['unknown',
                                                                              'illiterate',
                                                                              'basic',
                                                                              'high.school',
                                                                              'university.degree',
                                                                              'professional.course'],
                                                                             ['blue-collar',
                                                                              'technician',
                                                                              'management',
                                                                              'services',
                                                                              'retired',
                                                                              'admin.',
                                                                              'housemaid',
                                                                              'un...
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [9, 3, 1, 2, 8]),
                                                 ('OneHot Encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [7, 4, 5, 6, 14])],
                                   verbose_feature_names_out=False)),
                ('Numerical Transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scaling Numerical features',
                                                  StandardScaler(),
                                                  [0, 10, 11, 12, 13, 15, 16,
                                                   17, 18, 19])],
                                   verbose_feature_names_out=False))])

In [29]:
import joblib 

In [30]:
joblib.dump(pipe, '../model_dir/Preprocessor/Preprocessor.joblib')

['../model_dir/Preprocessor/Preprocessor.joblib']

--------------------------------------------------------------------------------------------------------------------------------

In [31]:
test_pipe = joblib.load('../model_dir/Preprocessor/Preprocessor.joblib')
test_pipe

Pipeline(steps=[('categorical Transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal Encoder',
                                                  OrdinalEncoder(categories=[['thu',
                                                                              'fri',
                                                                              'tue',
                                                                              'mon',
                                                                              'wed'],
                                                                             ['unknown',
                                                                              'illiterate',
                                                                              'basic',
                                                                              'high.school',
                                                                              'university.degree',
                                                                              'professional.course'],
                                                                             ['blue-collar',
                                                                              'technician',
                                                                              'management',
                                                                              'services',
                                                                              'retired',
                                                                              'admin.',
                                                                              'housemaid',
                                                                              'un...
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [9, 3, 1, 2, 8]),
                                                 ('OneHot Encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [7, 4, 5, 6, 14])],
                                   verbose_feature_names_out=False)),
                ('Numerical Transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scaling Numerical features',
                                                  StandardScaler(),
                                                  [0, 10, 11, 12, 13, 15, 16,
                                                   17, 18, 19])],
                                   verbose_feature_names_out=False))])

In [32]:

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [33]:
X.shape

(41188, 20)

In [34]:
X.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,210,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,138,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,339,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6
3,39,services,married,high.school,no,no,no,cellular,apr,fri,185,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,137,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2


In [35]:
e = X.head(1).copy()

In [36]:
e['education'] = 'basic'

In [37]:
e

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,44,blue-collar,married,basic,unknown,yes,no,cellular,aug,thu,210,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1


In [38]:
transformed_x = test_pipe.transform(e)

In [39]:
[transformed_x.info()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   day_of_week           1 non-null      float64
 1   housing_no            1 non-null      float64
 2   housing_unknown       1 non-null      float64
 3   housing_yes           1 non-null      float64
 4   loan_no               1 non-null      float64
 5   loan_yes              1 non-null      float64
 6   poutcome_failure      1 non-null      float64
 7   poutcome_nonexistent  1 non-null      float64
 8   poutcome_success      1 non-null      float64
 9   age                   1 non-null      float64
 10  education             1 non-null      float64
 11  job                   1 non-null      float64
 12  marital               1 non-null      float64
 13  month                 1 non-null      float64
 14  contact_cellular      1 non-null      float64
 15  contact_telephone     1 non

[None]

In [40]:
transformed_x.head()

,day_of_week,housing_no,housing_unknown,housing_yes,loan_no,loan_yes,poutcome_failure,poutcome_nonexistent,poutcome_success,age,...,loan_unknown,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,-1.400817,-0.908418,-0.156933,0.953401,0.461731,-0.422872,-0.33929,0.397706,-0.1857,0.381527,...,0.0,210,1,999,0,1.4,93.444,-36.1,4.963,5228.1
